In [6]:
import requests
import pandas as pd

# Read API key from .txt file
with open('../../Papers/mode choice/TRB2024/HHTS2012/codes_biogeme/API.txt', 'r') as f:
    api_key = f.read().strip()

#### OD Matrix creation

In [7]:
# Read location data from CSV
data1=pd.read_excel('ODcostmatrix.xlsx',sheet_name ='ODcostmatrix',index_col=0)
data1

,lat,long,remarks
home,41.751260,-111.809956,home-aggie vlg
school,41.728844,-111.841734,school-logan HS
work,41.839279,-111.855257,work-smithfield temple
lunch,41.718457,-111.838359,lunch- chipotle
grocery,41.718518,-111.876983,grocery-walmart
cinema,41.753205,-111.832208,cinema-megaplex theatre
amusement,41.723534,-111.852157,amusement-acqatic center


In [8]:
base_url = "http://dev.virtualearth.net/REST/v1/Routes/DistanceMatrix?"

locations = ";".join([f"{lat},{long}" for lat, long in zip(data1['lat'], data1['long'])])

url = f"{base_url}origins={locations}&destinations={locations}&travelMode=driving&key={api_key}"

response = requests.get(url)
result = response.json()

distance_matrix = result['resourceSets'][0]['resources'][0]['results']

cost_matrix = pd.DataFrame(index=data1.index, columns=data1.index)

for i in range(len(data1)):
    for j in range(len(data1)):
        cost_matrix.iloc[i, j] = distance_matrix[i * len(data1) + j]['travelDistance']/1.6

cost_matrix

,home,school,work,lunch,grocery,cinema,amusement
home,0.0,3.565,8.730625,4.62625,5.90125,1.6125,4.44375
school,3.568125,0.0,11.36,1.78375,2.774375,2.123125,0.954375
work,9.001875,11.32875,0.0,12.7475,12.080625,7.200625,11.2225
lunch,4.5925,1.438125,12.78625,0.0,2.921875,3.1475,1.724375
grocery,5.79375,2.66625,12.01125,2.81375,0.0,4.69625,1.683125
cinema,1.543125,2.123125,7.200625,3.184375,4.80375,0.0,3.00125
amusement,4.446875,1.01875,11.26125,1.441875,1.81375,3.00125,0.0


### Mode Specific Travel time in a single Row

In [9]:
# # Read location data from CSV
# data2=pd.read_excel('ODcostmatrix.xlsx',sheet_name ='self_made_data',index_col=0)
# data2.head()

,per_id,trip_id,origin,destination,o_lat,o_long,d_lat,d_long,mode_choice
hh_id,,,,,,,,,
1,1,1,home,school,41.751260,-111.809956,41.728844,-111.841734,car
1,1,2,school,work,41.728844,-111.841734,41.839279,-111.855257,car
1,1,3,work,lunch,41.839279,-111.855257,41.718457,-111.838359,transit
1,1,4,work,grocery,41.839279,-111.855257,41.718518,-111.876983,car
1,1,5,grocery,home,41.718518,-111.876983,41.751260,-111.809956,car


In [10]:
# # Function to get trip time for a specific mode
# def get_trip_time(origin_lat, origin_long, dest_lat, dest_long, mode_choice, departure_time=None):
#     base_url = 'https://dev.virtualearth.net/REST/V1/Routes'
#     params = {
#         'wp.0': f'{origin_lat},{origin_long}',
#         'wp.1': f'{dest_lat},{dest_long}',
#         'key': api_key,
#     }

#     if mode_choice == 'transit':
#         base_url += '/Transit'
#         if departure_time is not None:
#             params['dateTime'] = departure_time
#             params['timeType'] = 'Departure'

#     elif mode_choice == 'walking':
#         base_url += '/Walking'

#     elif mode_choice == 'bike':
#         base_url += '/Walking'

#     else:
#         base_url += '/Driving'

#     response = requests.get(base_url, params=params)
#     data = response.json()

#     # Extract the trip duration from the response
#     if mode_choice == 'transit':
#         trip_duration = data['resourceSets'][0]['resources'][0]['travelDuration']

#     elif mode_choice == 'walking':
#         trip_duration = data['resourceSets'][0]['resources'][0]['travelDuration']
    
#     elif mode_choice=='bike':
#         trip_duration = data['resourceSets'][0]['resources'][0]['travelDuration']

#     else:
#         trip_duration = data['resourceSets'][0]['resources'][0]['routeLegs'][0]['travelDuration']

#     return trip_duration

In [11]:
# # Iterate through each row and get trip time for each mode
# for index, row in data2.iterrows():
#     origin_lat = row['o_lat']
#     origin_long = row['o_long']
#     dest_lat = row['d_lat']
#     dest_long = row['d_long']
#     mode_choice = row['mode_choice']

#     if mode_choice == 'transit':
#     # Specify the departure_time for transit mode (replace '2023-07-20T08:00:00' with your desired time)
#         departure_time = '2023-07-20T08:00:00'
#         transit_time = get_trip_time(origin_lat, origin_long, dest_lat, dest_long, 'transit', departure_time)
#         travel_time = transit_time / 60
    
#     elif mode_choice == 'walking':
#         walk_time = get_trip_time(origin_lat, origin_long, dest_lat, dest_long, 'walking')
#         travel_time = walk_time / 60
    
#     elif mode_choice == 'bike':
#         bike_time = get_trip_time(origin_lat, origin_long, dest_lat, dest_long, 'bike')
#         travel_time = bike_time / (3 * 60)
    
#     else:
#         car_time = get_trip_time(origin_lat, origin_long, dest_lat, dest_long, 'car')
#         travel_time = car_time / 60
    
#     data2.at[index, 'travel_time'] = travel_time

# # Print the results
# print(data2)

### OD matrix CVTD Blue loop

In [12]:
# Read location data from CSV
data2=pd.read_excel('ODcostmatrix.xlsx',sheet_name ='CVTD_blueloop_stop',index_col=0)
data2.head()

,stop location,"originx,y","destx,y",mode_choice
stops ID,,,,
S1,150 e 442n logan,"41.740468284211204,-111.83060633077832","41.75550157391229,-111.83429513755848",car
S2,750 north main logan,"41.74569409085381,-111.83450739487982","41.76097994847846,-111.82898766174307",transit
S3,1080 north main logan,"41.75204384451957,-111.83447340194279","41.76131986796281,-111.83187376322584",walking
S4,1300 north main logan,"41.75550157391229,-111.83429513755848","41.756219240862166,-111.83451792433962",carpool
S5,1581 n 200e logan,"41.76097994847846,-111.82898766174307","41.751618478746316,-111.83456855773433",car


In [13]:
# Split the 'x,y' column into 'Latitude' and 'Longitude' columns
data2[['o_lat', 'o_long']] = data2['originx,y'].str.split(',', expand=True)
data2[['d_lat', 'd_long']] = data2['destx,y'].str.split(',', expand=True)
data2.head()

,stop location,"originx,y","destx,y",mode_choice,o_lat,o_long,d_lat,d_long
stops ID,,,,,,,,
S1,150 e 442n logan,"41.740468284211204,-111.83060633077832","41.75550157391229,-111.83429513755848",car,41.740468284211204,-111.83060633077832,41.75550157391229,-111.83429513755848
S2,750 north main logan,"41.74569409085381,-111.83450739487982","41.76097994847846,-111.82898766174307",transit,41.74569409085381,-111.83450739487982,41.76097994847846,-111.82898766174307
S3,1080 north main logan,"41.75204384451957,-111.83447340194279","41.76131986796281,-111.83187376322584",walking,41.75204384451957,-111.83447340194279,41.76131986796281,-111.83187376322584
S4,1300 north main logan,"41.75550157391229,-111.83429513755848","41.756219240862166,-111.83451792433962",carpool,41.75550157391229,-111.83429513755848,41.756219240862166,-111.83451792433962
S5,1581 n 200e logan,"41.76097994847846,-111.82898766174307","41.751618478746316,-111.83456855773433",car,41.76097994847846,-111.82898766174307,41.751618478746316,-111.83456855773433


In [14]:
columns_to_drop = ['originx,y', 'destx,y']
data2.drop(columns_to_drop, axis=1, inplace=True)
data2.head()

,stop location,mode_choice,o_lat,o_long,d_lat,d_long
stops ID,,,,,,
S1,150 e 442n logan,car,41.740468284211204,-111.83060633077832,41.75550157391229,-111.83429513755848
S2,750 north main logan,transit,41.74569409085381,-111.83450739487982,41.76097994847846,-111.82898766174307
S3,1080 north main logan,walking,41.75204384451957,-111.83447340194279,41.76131986796281,-111.83187376322584
S4,1300 north main logan,carpool,41.75550157391229,-111.83429513755848,41.756219240862166,-111.83451792433962
S5,1581 n 200e logan,car,41.76097994847846,-111.82898766174307,41.751618478746316,-111.83456855773433


OD Matrix

In [15]:
# base_url = "http://dev.virtualearth.net/REST/v1/Routes/DistanceMatrix?"

# locations = ";".join([f"{lat},{long}" for lat, long in zip(data2['o_lat'], data2['o_long'])])

# url = f"{base_url}origins={locations}&destinations={locations}&travelMode=driving&key={api_key}"

# response = requests.get(url)
# result = response.json()

# distance_matrix = result['resourceSets'][0]['resources'][0]['results']

# cost_matrix = pd.DataFrame(index=data2.index, columns=data2.index)

# for i in range(len(data2)):
#     for j in range(len(data2)):
#         cost_matrix.iloc[i, j] = distance_matrix[i * len(data2) + j]['travelDistance']/1.6   # Distance in miles

# cost_matrix      # Bing Map API has 25*25 OD matrix limitation

#### Travel time for each mode choices for each choice sets

In [16]:
# # Travel time for each mode choices for each choice sets
# def get_trip_time(origin_lat, origin_long, dest_lat, dest_long, mode_choice, departure_time=None):
#     base_url = 'https://dev.virtualearth.net/REST/V1/Routes'
#     params = {
#         'wp.0': f'{origin_lat},{origin_long}',
#         'wp.1': f'{dest_lat},{dest_long}',
#         'key': api_key,
#     }

#     if mode_choice == 'transit':
#         base_url += '/Transit'
#         if departure_time is not None:
#             params['dateTime'] = departure_time
#             params['timeType'] = 'Departure'

#     elif mode_choice == 'walking':
#         base_url += '/Walking'

#     elif mode_choice == 'carpool':
#         base_url += '/Driving'

#     else:
#         base_url += '/Driving'

#     response = requests.get(base_url, params=params)
#     data = response.json()

#     # Extract the trip duration from the response
#     if mode_choice == 'transit':
#         trip_duration = data['resourceSets'][0]['resources'][0]['travelDuration']

#     elif mode_choice == 'walking':
#         trip_duration = data['resourceSets'][0]['resources'][0]['travelDuration']

#     else:
#         trip_duration = data['resourceSets'][0]['resources'][0]['routeLegs'][0]['travelDuration']

#     return trip_duration



# # Add new columns for trip times
# data2['car_time'] = None
# data2['carpool_time'] = None
# data2['transit_time'] = None
# data2['walk_time'] = None

# # Iterate through each row and get trip time for each mode
# for index, row in data2.iterrows():
#     origin_lat = row['o_lat']
#     origin_long = row['o_long']
#     dest_lat = row['d_lat']
#     dest_long = row['d_long']

#     car_time = get_trip_time(origin_lat, origin_long, dest_lat, dest_long, 'car')
#     data2.at[index, 'car_time'] = car_time / 60

#     carpool_time = get_trip_time(origin_lat, origin_long, dest_lat, dest_long, 'carpool')
#     data2.at[index, 'carpool_time'] = carpool_time * 1.20 / 60

#     # Specify the departure_time for transit mode (replace '2023-07-20T08:00:00' with your desired time)
#     departure_time = '2023-07-20T08:00:00'
#     transit_time = get_trip_time(origin_lat, origin_long, dest_lat, dest_long, 'transit', departure_time)
#     data2.at[index, 'transit_time'] = transit_time / 60

#     walk_time = get_trip_time(origin_lat, origin_long, dest_lat, dest_long, 'walking')
#     data2.at[index, 'walk_time'] = walk_time / 60

# # Print the results
# print(data2)

                      stop location mode_choice               o_lat  \
stops ID                                                              
S1                 150 e 442n logan         car  41.740468284211204   
S2             750 north main logan     transit   41.74569409085381   
S3            1080 north main logan     walking   41.75204384451957   
S4            1300 north main logan     carpool   41.75550157391229   
S5                1581 n 200e logan         car   41.76097994847846   
S6                1600 n 100e logan     transit   41.76131986796281   
S7            1341 north main logan     walking  41.756219240862166   
S8            1045 north main logan     carpool  41.751618478746316   
S9             745 north main logan         car  41.745524532448805   
S10            355 north main logan     transit  41.737959470262616   
S11             55 north main logan     walking  41.732299100119846   
S12            359 south main logan     carpool   41.72448424762579   
S13   

### Mode specific origin destination travel time for space time prism

In [17]:
# Function to get trip time for a specific mode
def get_trip_time(origin_lat, origin_long, dest_lat, dest_long, mode_choice, departure_time=None):
    base_url = 'https://dev.virtualearth.net/REST/V1/Routes'
    params = {
        'wp.0': f'{origin_lat},{origin_long}',
        'wp.1': f'{dest_lat},{dest_long}',
        'key': api_key,
    }

    if mode_choice == 'transit':
        base_url += '/Transit'
        if departure_time is not None:
            params['dateTime'] = departure_time
            params['timeType'] = 'Departure'

    elif mode_choice == 'walking':
        base_url += '/Walking'

    else:
        base_url += '/Driving'

    response = requests.get(base_url, params=params)
    data = response.json()

    # Extract the trip duration from the response
    if mode_choice == 'transit':
        trip_duration = data['resourceSets'][0]['resources'][0]['travelDuration']

    elif mode_choice == 'walking':
        trip_duration = data['resourceSets'][0]['resources'][0]['travelDuration']
    
    elif mode_choice=='carpool':
        trip_duration = data['resourceSets'][0]['resources'][0]['routeLegs'][0]['travelDuration']

    else:
        trip_duration = data['resourceSets'][0]['resources'][0]['routeLegs'][0]['travelDuration']

    return trip_duration

In [18]:
# Iterate through each row and get trip time for each mode
for index, row in data2.iterrows():
    origin_lat = row['o_lat']
    origin_long = row['o_long']
    dest_lat = row['d_lat']
    dest_long = row['d_long']
    mode_choice = row['mode_choice']

    if mode_choice == 'transit':
    # Specify the departure_time for transit mode (replace '2023-07-20T08:00:00' with your desired time)
        departure_time = '2023-07-20T08:00:00'
        transit_time = get_trip_time(origin_lat, origin_long, dest_lat, dest_long, 'transit', departure_time)
        travel_time = transit_time / 60
    
    elif mode_choice == 'walking':
        walk_time = get_trip_time(origin_lat, origin_long, dest_lat, dest_long, 'walking')
        travel_time = walk_time / 60
    
    elif mode_choice == 'car':
        car_time = get_trip_time(origin_lat, origin_long, dest_lat, dest_long, 'car')
        travel_time = car_time / 60
    
    else:
        carpool_time = get_trip_time(origin_lat, origin_long, dest_lat, dest_long, 'carpool')
        travel_time = carpool_time * 1.2 / 60
    
    data2.at[index, 'travel_time'] = travel_time

# Print the results
print(data2)

                      stop location mode_choice               o_lat  \
stops ID                                                              
S1                 150 e 442n logan         car  41.740468284211204   
S2             750 north main logan     transit   41.74569409085381   
S3            1080 north main logan     walking   41.75204384451957   
S4            1300 north main logan     carpool   41.75550157391229   
S5                1581 n 200e logan         car   41.76097994847846   
S6                1600 n 100e logan     transit   41.76131986796281   
S7            1341 north main logan     walking  41.756219240862166   
S8            1045 north main logan     carpool  41.751618478746316   
S9             745 north main logan         car  41.745524532448805   
S10            355 north main logan     transit  41.737959470262616   
S11             55 north main logan     walking  41.732299100119846   
S12            359 south main logan     carpool   41.72448424762579   
S13   